# HuggingFace Fine-Tuning NER model

In [1]:
from pathlib import Path
import re
import sys
import random

import numpy as np
from sklearn.model_selection import train_test_split
import torch

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import Trainer
from transformers import TrainingArguments

from datasets import load_metric

from spacy import displacy

print('* loaded all libs')

/infodev1/non-phi-projects/huan/env/nlpy37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* loaded all libs


## Pre-processing the dataset

In [2]:
dataset_fn = 'dataset.tsv'
test_size = 0.2
model_name = 'distilbert-base-cased'
fine_tuned_model_path = "./fine-tuned-model/covid19_symp_model"
output_dir = './results'

In [3]:
def read_dataset(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs


def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# parse the texts and tags
texts, tags = read_dataset(dataset_fn)

# split into train val
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=test_size)

# Next, let’s create encodings for our tokens and tags. 
# For the tags, we can start by just create a simple mapping which we’ll use in a moment:
unique_tags = set(tag for doc in tags for tag in doc)
label_list = list(unique_tags)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

print('* train texts: ', len(train_texts))
print('* unique tags: ', len(unique_tags))
print('* tags: ', label_list)

# get encodings
train_encodings = tokenizer(
    train_texts, 
    is_split_into_words=True, 
    return_offsets_mapping=True, 
    padding=True, 
    truncation=True
)
val_encodings = tokenizer(
    val_texts, 
    is_split_into_words=True, 
    return_offsets_mapping=True, 
    padding=True, 
    truncation=True
)

# get labels
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

# get datasets
# we don't want to pass this to the model
train_encodings.pop("offset_mapping") 
val_encodings.pop("offset_mapping")

train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)

print('* created dataset for training')

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 29.0/29.0 [00:00<00:00, 14.2kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:00<00:00, 302kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 213k/213k [00:00<00:00, 2.01MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 436k/436k [00:00<00:00, 3.03MB/s]


* train texts:  129
* unique tags:  5
* tags:  ['B-SYMP', 'O', 'I-TREATMENT', 'I-SYMP', 'B-TREATMENT']
* created dataset for training


## Fine-tuning the model

In [7]:
training_args = TrainingArguments(
    output_dir=output_dir,           # output directory
    num_train_epochs=50,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModelForTokenClassification.from_pretrained(
    model_name, 
    num_labels=len(unique_tags), 
    id2label=id2tag, 
    label2id=tag2id
)

metric = load_metric("seqeval")
return_entity_level_metrics = True
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()
print(trainer.evaluate())

# save the model
model.save_pretrained(fine_tuned_model_path)
print('* done fine-tuning and saved the fine-tuned model')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /home/m210842/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "B-SYMP",
    "1": "O",
    "2": "I-TREATMENT",
    "3": "I-SYMP",
    "4": "B-TREATMENT"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-SYMP": 0,
    "B-TREATMENT": 4,
    "I-SYMP": 3,
    "I-TREATMENT": 2,
    "O": 1
  },
  "max_position_embe

Step,Training Loss
10,1.430400
20,1.358200
30,1.255700
40,1.101300
50,0.923700
60,0.732300
70,0.554600
80,0.486000
90,0.317600
100,0.355800




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 33
  Batch size = 256


Configuration saved in ./fine-tuned-model/covid19_symp_model/config.json


{'eval_loss': 0.21623656153678894, 'eval_SYMP_precision': 0.6451612903225806, 'eval_SYMP_recall': 0.7547169811320755, 'eval_SYMP_f1': 0.6956521739130435, 'eval_SYMP_number': 53, 'eval_TREATMENT_precision': 0.8787878787878788, 'eval_TREATMENT_recall': 0.9354838709677419, 'eval_TREATMENT_f1': 0.90625, 'eval_TREATMENT_number': 31, 'eval_overall_precision': 0.7263157894736842, 'eval_overall_recall': 0.8214285714285714, 'eval_overall_f1': 0.7709497206703911, 'eval_overall_accuracy': 0.9602510460251046, 'eval_runtime': 0.1339, 'eval_samples_per_second': 246.492, 'eval_steps_per_second': 7.469, 'epoch': 100.0}


Model weights saved in ./fine-tuned-model/covid19_symp_model/pytorch_model.bin


* done fine-tuning and saved the fine-tuned model


## Predict

In [8]:
# if we have a pre-trained model we could just load it
model = AutoModelForTokenClassification.from_pretrained(fine_tuned_model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('* loaded the fine-turned model and tokenizer')

loading configuration file ./fine-tuned-model/covid19_symp_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "B-SYMP",
    "1": "O",
    "2": "I-TREATMENT",
    "3": "I-SYMP",
    "4": "B-TREATMENT"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-SYMP": 0,
    "B-TREATMENT": 4,
    "I-SYMP": 3,
    "I-TREATMENT": 2,
    "O": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "vocab_size": 28996
}

loading weights file ./fine-tuned-model/covid19_symp_model/pytorch_model.bin
Al

* loaded the fine-turned model and tokenizer


In [9]:
# the default result doesn't group the entities
# so we need to specify the `aggregation_strategy` to group the entity for display
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='first')

texts = [
    "I was feeling bad all over about 7 pm aching and chills all over, and got nausea and extreme headache",
    "Moderna Covid-19 vaccine EUA Soreness at the injection site, body aches, chills, fatigue Treated effectively with ibuprofen",
    "Lightheaded immediately and almost passed out.  Felt a tingling in both hands.",
    "Metallic taste in mouth for about an hour.  8 hours later had the chills, body aches, headache, sinus pressure, and blocked ears.",
    "I feel head is ache and have some kind of breathing issue, but I don't take any aspirin"
]

# for showing the tags
get_color = lambda: "#"+''.join([random.choice('89ABCDEF') for j in range(6)])

# for spacy display
options = {"ents": [], "colors": {}}
for label in label_list:
    if label == 'O': continue
    ent = label[2:]
    if ent in options['ents']: continue
        
    options['ents'].append(ent)
    color = "linear-gradient(90deg, %s, %s)" % (get_color(), get_color())
    options['colors'][ent] = color

# predict each text
for i,text in enumerate(texts):
    result = nlp(text)
    doc = {
        "text": text,
        "ents": []
    }
    for ent in result:
        doc['ents'].append({
            'start': ent['start'], 
            'end': ent['end'], 
            'label': ent['entity_group']
        })
        
    displacy.render([doc], style='ent', manual=True, options=options)
    print('-'*60)

------------------------------------------------------------


------------------------------------------------------------


------------------------------------------------------------


------------------------------------------------------------


------------------------------------------------------------
